# Рекомендация тарифов

В данной работе было необходимо построить модель для задачи классификации, которая выберет подходящий тариф для клиента. В распоряжении были данные о поведении клиентов. 
Условие для выполнения работы: *accuracy* на тестовой выборке должна быть не меньше 0.75.


Ход работы:
* Откроем файл с данными и изучим его.
* Разделим исходные данные на обучающую, валидационную и тестовую выборки.
* Исследуем качество разных моделей, меняя гиперпараметры. Кратко напишем выводы исследования.
* Проверим качество модели на тестовой выборке.
* Дополнительное задание: проверить модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

## Откройте и изучите файл

Импортируем библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.dummy import DummyClassifier

Прочитаем файл с данными

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на наши данные:

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

Наши данные разобьём на обучающую, валидационную и тестовую выборки в соотношении: 6/2/2.

In [5]:
# отделяем обучающую выборку
df_train, df_temporary = train_test_split(df, test_size=0.4, random_state=12345)

In [6]:
# разделим тестовую и валидационную выборки
df_test, df_valid = train_test_split(df_temporary, test_size=0.5, random_state=12345)

In [7]:
# проверим размер наших выборок
print('Длина наших выборок в % от общего количества данных')
print(f'Обучающей: {len(df_train) / len(df):.0%}, валидационной: {len(df_valid) / len(df):.0%}, тестовой: {len(df_test) / len(df):.0%}')

Длина наших выборок в % от общего количества данных
Обучающей: 60%, валидационной: 20%, тестовой: 20%


Теперь наши выборки готовы. Прежде чем переходить к обучению, разделим целевой признак наших данных от остальных.

In [8]:
features_train = df_train.drop('is_ultra', axis = 1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis = 1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop('is_ultra', axis = 1)
target_test = df_test['is_ultra']

## Исследуйте модели

В этом разделе исследуем качество разных моделей. Будем менять значения гиперпараметров и смотреть, как меняется количество правильных ответов.

Для начала обучим модель, не указав ни одного гиперпараметра и посмотрим на количество правильных ответов.

In [9]:
model = DecisionTreeClassifier()

In [10]:
# обучаем нашу модель
model.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [11]:
# рассчитаем значение Accuracy
accuracy_score(target_valid, model.predict(features_valid))

0.7325038880248833

73% правильных ответов. Давайте попробуем улучшить этот результат. Будем менять значение гиперпараметра 'max_depth' (глубина дерева) и смотреть, модель с каким значением выдает лучший результат. Переберать значения будем от 1 до 10.

In [12]:
best_result = 0
best_model = None
best_depth = 0
for depth in range(1, 11):
    model_depth = DecisionTreeClassifier(random_state=23567, max_depth=depth)
    model_depth.fit(features_train, target_train)
    result = accuracy_score(target_valid, model_depth.predict(features_valid))
    if result > best_result:
        best_results = result
        best_depth = depth
print(f'Лучший показатель Accuracy: {best_results:.2f} принадлежит модели со значением "max_depth": {best_depth}')

Лучший показатель Accuracy: 0.78 принадлежит модели со значением "max_depth": 10


Это ужа лучше, 78% правильных ответов. Но можно ли лучше? Чтобы узнать это проверим другую модель машинного обучения 'RandomForestClassifier', у которой будем менять гиперпараметр 'n_estimators', который отвечает за количество созданных деревьев. Перебирать значения будем от 1 до 10.

In [13]:
best_result = 0
best_model = None
best_depth = 0
for est in range(1,11):
    model_tree = RandomForestClassifier(random_state=23567, n_estimators=est)
    model_tree.fit(features_train, target_train)
    result = model_tree.score(features_valid, target_valid)
    if result > best_result:
        best_result = result
        best_est = est
print(f'Лучший показатель Accuracy: {best_result:.2f} принадлежит модели со значением "n_estimators": {best_est}')

Лучший показатель Accuracy: 0.79 принадлежит модели со значением "n_estimators": 10


Ещё 1%, класс. Похоже, мы двигаемся в правильном направлении. Теперь давайте проверим третий тип модели машинного обучения логистическую регрессию.

In [14]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
model.score(features_valid, target_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7402799377916018

### Вывод

Мы проверили 3 разные модели машинного обучения. Создавая модели, меняли гиперпараметры внутри. Выяснили, что лучше всего предсказывает значение тарифа модель машинного обучения типа случайного леса с количеством деревьев равным 10. Теперь давайте проверим эту модель на нашей тестовой выборке.

## Проверьте модель на тестовой выборке

In [15]:
model = RandomForestClassifier(random_state=23567, n_estimators=10)
model.fit(features_train, target_train)
accuracy_score(target_test, model.predict(features_test))

0.7713841368584758

### Вывод

Проверили нашу модель на тестовой выборке. Значение 'Accuracy' равно 0.77. Это показывает, что наше исследование мы провели не зря. В итоге самая лучшая модель для наших данных оказалась модель типа 'RandomForestClassifier' с количеством деревьев равным 10.

## (бонус) Проверьте модели на адекватность

Для того, чтобы проверить нашу модель на вменяемость воспользуемся классификатором DummyClassifier,который делает прогнозы, используя простые правила.

In [16]:
dummy = DummyClassifier(strategy="most_frequent").fit(features_train, target_train)
accuracy_score(target_test, dummy.predict(features_test))

0.7060653188180405

Видно, что значение 'Accuracy' в нашей модели больше, чем в модели, которая использует классиыикатор DummyClassifier. Значит можно сделать вывод, что нашу работу мы проделали не зря.